https://github.com/karndeepsingh/ApplicationsBuildWithLLMs/blob/main/Zero_Shot_NER_with_ChatGPT.ipynb

In [1]:
%pip install --upgrade openai

  Attempting uninstall: openai
    Found existing installation: openai 0.27.0
    Uninstalling openai-0.27.0:
      Successfully uninstalled openai-0.27.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
promptify 2.0.3 requires huggingface_hub==0.12, but you have huggingface-hub 0.26.3 which is incompatible.
promptify 2.0.3 requires openai==0.27.0, but you have openai 1.57.0 which is incompatible.
promptify 2.0.3 requires tqdm==4.65.0, but you have tqdm 4.67.1 which is incompatible.


In [2]:
import openai
import os
import re
import ast
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

openai.api_key = OPENAI_API_KEY

In [ ]:
SYSTEM_PROMPT = "You are a smart and intelligent Named Entity Recognition (NER) system. I will provide you the definition of the entities you need to extract and the sentence from which you need to extract the entities and the output in given format with examples."

USER_PROMPT_1 = "Are you clear about your role?"

ASSISTANT_PROMPT_1 = "Sure, I'm ready to help you with your NER task. Please provide me with the necessary information to get started."

PROMPT = (
    "Entity Definition:\n"
    "1. PERSON: Short name or full name of a person from any geographic regions.\n"
    "2. DATE: Any format of dates. Dates can also be in natural language.\n"
    "3. LOC: Name of any geographic location, like cities, countries, continents, districts etc.\n"
    "4. ORG: Name of the companies like Google, samsung, Apple etc.\n"
    "5. NUMBERS: Numerical entites which are numerically present or mentioned in words like 7000, half of dozen etc.\n"
    "6. WORK_OF_ART: Acronym or full name of Movie, Series like Spider-man, Venom ltbc etc. \n"
    "7. SPORTS: Acronym or full name of Game like cod, pubg etc. \n"
    "\n"
    "Output Format:\n"
    "{{'WORK_OF_ART': [list of entities present], 'SPORTS': [list of entities present], 'PERSON': [list of entities present], 'DATE': [list of entities present], 'LOC': [list of entities present],'ORG': [list of entities present],'NUMBERS': [list of entities present]}}\n"
    "If no entities are presented in any categories keep it None\n"
    "\n"
    "Examples:\n"
    "\n"
    "1. Sentence: USA and India are friends. G20 summit going to held in India in September 2023. Indian Prime Minister Narendra Modi will be hosting it and TATA will be giving charity of $150 Million.\n"
    "Output: {{'WORK_OF_ART': ['None'], 'SPORTS': ['None'], 'PERSON': ['Narendra Modi'], 'DATE': ['September 2023'], 'LOC': ['USA','India','India'],'ORG':['TATA'],'NUMBERS':['150 Million']}}\n"
    "\n"
    "2. Sentence: Mr.John and Sunita Roy are friends and they meet each other on 24/03/1998 in Samsung while they were co-workers and shared Rs.8000 in exchange for some work.\n"
    "Output: {{'WORK_OF_ART': ['None'], 'SPORTS': ['None'], 'PERSON': ['Mr. John', 'Sunita Roy'], 'DATE': ['24/03/1998'], 'LOC': ['None'],'ORG':['Samsung'],'NUMBERS':['8000']}}\n"
    "\n"
    "3. Sentence: Do you like Spider-man?\n"
    "Output: {{'WORK_OF_ART': ['Spider-man'], 'SPORTS': ['None'], 'PERSON': ['None'], 'DATE': ['None'], 'LOC': ['None'],'ORG':['None'],'NUMBERS':['None']}}\n"
    "\n"
    "4. Sentence: I love to play cod.\n"
    "Output: {{'WORK_OF_ART': ['None'], 'SPORTS': ['cod'], 'PERSON': ['Mr. John', 'Sunita Roy'], 'DATE': ['24/03/1998'], 'LOC': ['None'],'ORG':['Samsung'],'NUMBERS':['8000']}}\n"
    "\n"
    "5. Sentence: {}\n"
    "Output: "
)

In [4]:
def openai_chat_completion_response(final_prompt):
  response = openai.ChatCompletion.create(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": USER_PROMPT_1},
                    {"role": "assistant", "content": ASSISTANT_PROMPT_1},
                    {"role": "user", "content": final_prompt}
                ]
            )

  return response['choices'][0]['message']['content'].strip(" \n")

In [5]:
my_sentence = """William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor, and philanthropist. He is best known as the co-founder of Microsoft Corporation. During his career at Microsoft, Gates held the positions of chairman, chief executive officer (CEO), president and chief software architect, while also being the largest individual shareholder until May 2014. He is one of the best-known entrepreneurs and pioneers of the microcomputer revolution of the 1970s and 1980s. Born and raised in Seattle, Washington, Gates co-founded Microsoft with childhood friend Paul Allen in 1975, in Albuquerque, New Mexico; it went on to become the world's largest personal computer software company. Gates led the company as chairman and CEO until stepping down as CEO in January 2000, but he remained chairman and became chief software architect. During the late 1990s, Gates had been criticized for his business tactics, which have been considered anti-competitive. This opinion has been upheld by numerous court rulings. In June 2006, Gates announced that he would be transitioning to a part-time role at Microsoft and full-time work at the Bill & Melinda Gates Foundation, the private charitable foundation that he and his wife, Melinda Gates, established in 2000.[9] He gradually transferred his duties to Ray Ozzie and Craig Mundie. He stepped down as chairman of Microsoft in February 2014 and assumed a new post as technology adviser to support the newly appointed CEO Satya Nadella."""


In [ ]:
PROMPT = PROMPT.format(my_sentence)
ners = openai_chat_completion_response(PROMPT)
ners